In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import TransformerMixin
from sklearn import metrics

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("train:", train_df.shape)
print("test:" ,test_df.shape)

                                    #correlation matrix#
def corr_plot2(dataframe2, top_n2, target2, fig_x2, fig_y2):
    corrmat2 = dataframe2.corr()
    top_n2 = top_n2 + 1 
    cols2 = corrmat2.nlargest(top_n2, target2)[target2].index
    cm2 = np.corrcoef(train_df[cols2].values.T)
    f2, ax2 = plt.subplots(figsize=(fig_x2,fig_y2))
    sns.set(font_scale=1.25)
    cmap2 = plt.cm.viridis
    hm2 = sns.heatmap(cm2, cbar=False, annot=True, square=True,cmap = cmap2, fmt='.2f', annot_kws={'size': 10},	 yticklabels=cols2.values, xticklabels=cols2.values)
    plt.show()
    return cols2,cm2
def corr_plot(dataframe, top_n, target, fig_x, fig_y):
    corrmat = dataframe.corr()
    top_n = top_n + 1 
    cols = corrmat.nlargest(top_n, target)[target].index
    cm = np.corrcoef(train_df[cols].values.T)
    return cols,cm
corr_20,cm = corr_plot(train_df, 30, 'price_doc', 10,10) 
corr_22 = corr_plot2(train_df, 30, 'price_doc', 10,10)  



                           #Determine numerical columns and strings colums#
dtypes = train_df.iloc[:,2:-1].dtypes
string_cols = dtypes[dtypes == object].index
num_cols = dtypes[dtypes != object].index


                              #description of non_mumerical colones  #
def categorical_summary(data, col):
    filled_values = sum(data[col].notnull())
    missing = sum(data[col].isnull())
    cardinality = len(data[col].unique())
    print(col.upper())
    print('-------------------------------')
    print('filled count: %s' %filled_values)
    print('missing count: %s' %missing)
    print('cardinality: %s' %cardinality)
    print(data[col].value_counts())
for col in string_cols:
    categorical_summary(train_df, col)
    
    
                                  #"yes" and "no" transformation #
yes_no_list = ['culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion',
          'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion',
          'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line']
for col in yes_no_list:
    train_df.ix[train_df[col]=='yes', col]=1
    train_df.ix[train_df[col]=='no', col]=0
    test_df.ix[test_df[col]=='yes', col]=1
    test_df.ix[test_df[col]=='no', col]=0
    

                                   #ecology column transformation#
train_df.ix[train_df["ecology"]=='poor', "ecology"]=0
train_df.ix[train_df["ecology"]=='good', "ecology"]=2
train_df.ix[train_df["ecology"]=='excellent', "ecology"]=3
train_df.ix[train_df["ecology"]=='satisfactory', "ecology"]=1
test_df.ix[test_df["ecology"]=='poor', "ecology"]=0
test_df.ix[test_df["ecology"]=='good', "ecology"]=2
test_df.ix[test_df["ecology"]=='excellent', "ecology"]=3
test_df.ix[test_df["ecology"]=='satisfactory', "ecology"]=1


                     #determine the ecology's mean without missing data#
i = 0;
s = 0;
for lig in train_df["ecology"]:
    if lig != "no data" :
        i=i+1
        s=s+lig
mean = s/i   
print (mean)
i = 0;
s = 0;
for lig in test_df["ecology"]:
    if lig != "no data" :
        i=i+1
        s=s+lig
mean = s/i   
print (mean)


                  #replace the missing data with the mean of ecology#
train_df.ix[train_df["ecology"]=='no data', "ecology"]=1
test_df.ix[test_df["ecology"]=='no data', "ecology"]=1


                        #transformation of product_type#
train_df.ix[train_df["product_type"]=='Investment', "product_type"]=1
train_df.ix[train_df["product_type"]=='OwnerOccupier', "product_type"]=0
test_df.ix[test_df["product_type"]=='Investment', "product_type"]=1
test_df.ix[test_df["product_type"]=='OwnerOccupier', "product_type"]=0
print (test_df.head)


                          #test , train  's transformation# 
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
        if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
train_df = DataFrameImputer().fit_transform(train_df)
test_df = DataFrameImputer().fit_transform(test_df)



                                  #linear regression#
y = train_df.price_doc
train_X = train_df.drop(["price_doc","id","timestamp","sub_area"],axis=1)
linreg = LinearRegression()
linreg.fit(train_X, y)
print(linreg.intercept_)
print(linreg.coef_)
test_X = test_df.drop(["id","timestamp","sub_area"],axis=1)
y_pred = linreg.predict(test_X)
print (y_pred)
print (y_pred.shape)


                                       #submission#
submission_df = pd.DataFrame({'id':test_df.id, 'price_doc':y_pred}).set_index('id').to_csv('submission.csv')
cwd = os.getcwd()
#print (cwd)

